# **19_recommend**
- 협업 필터링
- Popularity, High Rated Based(가장 단순함)
- 가장 인기도(혹은 별점)을 갖는 item을 추천하는 방식
- 모두에게 동일한 item이 추천됨

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 데이터 불러오기
ratings = pd.read_csv('https://raw.githubusercontent.com/luxdolorosa/data_set/master/movie/ratings.csv', encoding = 'euckr', sep = '\t')

In [ ]:
ratings.head()

,no,name,title,rating
0,1,최트루,택시운전사,3.0
1,2,최트루,신과함께,4.0
2,3,최트루,공조,3.5
3,4,최트루,스파이더맨,5.0
4,5,최트루,범죄도시,2.0


In [ ]:
ratings.tail()

,no,name,title,rating
29,30,포세이돈,군함도,3.0
30,31,헤르메스,스파이더맨,3.0
31,32,헤르메스,범죄도시,2.5
32,33,헤르메스,택시운전사,3.5
33,34,헤르메스,공조,2.0


In [ ]:
ratings

,no,name,title,rating
0,1,최트루,택시운전사,3.0
1,2,최트루,신과함께,4.0
2,3,최트루,공조,3.5
3,4,최트루,스파이더맨,5.0
4,5,최트루,범죄도시,2.0
5,6,최트루,군함도,4.0
6,7,지리산,택시운전사,4.0
7,8,지리산,신과함께,5.0
8,9,지리산,공조,3.0
9,10,지리산,스파이더맨,2.0


In [ ]:
# 데이터 요약정보
ratings.describe()

,no,rating
count,34.000000,34.000000
mean,17.500000,3.397059
std,9.958246,1.005887
min,1.000000,2.000000
25%,9.250000,2.500000
50%,17.500000,3.500000
75%,25.750000,4.000000
max,34.000000,5.000000


In [ ]:
# 실습하기. 각 영화별 평점정보 더하기
ratings.groupby('title').sum()

,no,rating
title,,
공조,109,19.5
군함도,90,16.5
범죄도시,117,14.5
스파이더맨,111,20.5
신과함께,70,24.0
택시운전사,98,20.5


In [ ]:
# 가장 높은 인기도를 가진 item 1개를 추천하기
df = ratings.groupby('title').sum()
df.loc[:, 'rating'].max()

24.0

In [ ]:
df.loc[df.rating == df.loc[:, 'rating'].max(), :].head(1)

,no,rating
title,,
신과함께,70,24.0


# **협업필터링**
- Collaborative Filtering
- 사용자가 입력한 선호도(평점)을 바탕으로 다른 사용자와의 유사도를 비교하여 추천을 받음
- 사용자의 데이터가 없는(cold start)가 어려움

In [ ]:
# 데이터 불러오기
ratings = pd.read_csv('https://raw.githubusercontent.com/luxdolorosa/data_set/master/movie/ratings.csv', encoding = 'euckr', sep = '\t', index_col = 0)

In [ ]:
ratings.head()

,name,title,rating
no,,,
1,최트루,택시운전사,3.0
2,최트루,신과함께,4.0
3,최트루,공조,3.5
4,최트루,스파이더맨,5.0
5,최트루,범죄도시,2.0


In [ ]:
# 리뷰어별로 각 영화에 평점을 어떻게 주었는가로 데이터의 모양을 변경하기
# - 피벗 이용하기
movie_ratings = pd.pivot(ratings, index = 'title', columns = 'name', values = 'rating')

In [ ]:
# 상관도 구하기
movie_ratings.corr()
sim_user = movie_ratings.corr().reset_index()
sim_user

name,name,조쌤,지리산,최트루,코리아,포세이돈,헤르메스
0,조쌤,1.000000,0.151247,0.440589,0.600081,0.367410,-0.563602
1,지리산,0.151247,1.000000,-0.184000,0.616259,0.754726,0.316228
2,최트루,0.440589,-0.184000,1.000000,0.464565,0.388354,0.154919
3,코리아,0.600081,0.616259,0.464565,1.000000,0.633920,-0.075593
4,포세이돈,0.367410,0.754726,0.388354,0.633920,1.000000,0.946729
5,헤르메스,-0.563602,0.316228,0.154919,-0.075593,0.946729,1.000000


In [ ]:
# 헤르메스가 평가하지 않은 영화를 골라내기
tmp_title = movie_ratings.loc[:, '헤르메스']
tmp_title.isnull()

na_title = list(movie_ratings[tmp_title.isnull()].index)
na_title

['군함도', '신과함께']

In [ ]:
# 영화평점 데이터 중 헤르메스가 평가하지 않은 영화의 평점 조회하기
ratings.apply(lambda x: x.title in na_title, axis = 1)
ratings_t = ratings[ratings.apply(lambda x: x.title in na_title, axis = 1)]

In [ ]:
# 헤르메스의 유사도 정보 조회하기
s = sim_user.loc[:, ['name', '헤르메스']]
s

name,name,헤르메스
0,조쌤,-0.563602
1,지리산,0.316228
2,최트루,0.154919
3,코리아,-0.075593
4,포세이돈,0.946729
5,헤르메스,1.000000


In [ ]:
# 헤르메스와의 상관도 추가하기
recommend = pd.merge(ratings_t, s)
# 헤르메스 열의 이름을 similarity로 변경하기
recommend.rename(columns = {'헤르메스':'similarity'}, inplace = True)

In [ ]:
# 실습하기. 헤르메스와 가장 유사도가 높은 리뷰어가 추천하는 영화 1개를 조회해주세요.
recommend.sort_values(['similarity', 'rating'], ascending = False).iloc[0, 1]

'신과함께'